In [13]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
from pprint import pprint
from pathlib import Path
from random import randint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from tqdm.notebook import tqdm
import nibabel as nib
import glmsingle
from glmsingle.glmsingle import GLM_single
import bids
from bids import BIDSLayout
from scipy.ndimage import zoom

dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

from glmsingle_utils import (
    load_results, 
    plot_results,
    save_results
)
from kamitani import (
    load_data,
    make_design_matrix
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Enter the path to the Kamitani dataset, example: X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\fmriprep-20.2.4
dataset_path = Path(input())

 X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\fmriprep-20.2.4


In [4]:
# Load layouts, this will take about a minute because pybids is slow

t = time.time()
dataset_layout = bids.BIDSLayout(dataset_path)
print(time.time() - t)

66.3084704875946


In [18]:
fmri_batch[0].dtype

dtype('float64')

In [15]:
subject = '02' # ['01', '02', '03']
space = 'T1w'
run_ids = list(range(1, 9))
sessions = [f'perceptionNaturalImageTraining{i:02}' for i in range(1, 16)]

fmri_batch, events_dfs, fmri_mask = load_data(dataset_layout, subject, sessions, run_ids=run_ids)

  7%|███████████████▏                                                                                                                                                                                                                    | 8/120 [00:54<12:39,  6.78s/it]


KeyboardInterrupt: 

In [8]:
# Load the fmri mask. 
# fmriPrep outputs one mask for every functional run, but they are mostly identical
# This will grab the mask from the very first one
mask_image = dataset_layout.get(
    subject=subject, 
    session='perceptionNaturalImageTraining01', 
    space=space, 
    run=1, 
    desc='brain', 
    extension='nii.gz'
)[0].get_image()
fmri_mask = mask_image.get_fdata().astype(bool)
H, W, D = fmri_mask.shape

# Load the fmri data and apply the mask
fmri_batch = []
for run_image in tqdm(run_images):
    fmri_data = run_image.get_fdata()
    fmri_data = fmri_data[fmri_mask]
    fmri_batch.append(fmri_data)

  0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
# Cell for visualizing the fMRI runs
all_runs = np.concatenate(fmri_batch)
mean, std = all_runs.mean(), all_runs.std()

show_volume = np.zeros_like(fmri_mask, dtype=float)
_, T = fmri_batch[0].shape

@interact(run_id=(0, len(fmri_batch)-1), d=(0, D-1), t=(0, T-1))
def show(run_id, d, t):
    show_volume[fmri_mask] = fmri_batch[run_id][:, t]
    plt.figure(figsize=(8, 8))
    plt.imshow(show_volume[:, :, d], cmap='gray', vmin=mean-2*std, vmax=mean+2*std)
    plt.show()

interactive(children=(IntSlider(value=2, description='run_id', max=4), IntSlider(value=37, description='d', ma…

In [18]:
# Create the design matrices

# Collect all conditions
conditions = []
for events_df in events_dfs:
    for i, event in events_df.iterrows():
        if int(event['event_type']) != 1:
            continue
        conditions.append(event['stimulus_id'])
conditions = list(set(conditions))
conditions.sort()
conditions = {condition: i  for i, condition in enumerate(conditions)}
C = len(conditions)

T = fmri_batch[0].shape[-1]

# Construct the design matrices
design_batch = []
for run_id, events_df in enumerate(events_dfs):
    design_matrix = np.zeros(shape=(T, C))
    
    for i, event in events_df.iterrows():
        if int(event['event_type']) != 1:
            continue
        condition_name = event['stimulus_id']
        c = conditions[condition_name]
        t = round(event['onset'] / tr)
        design_matrix[t, c] = 1
    design_batch.append(design_matrix)
    
@interact(run_id=(0, len(design_batch)-1))
def show(run_id):
    design_matrix = design_batch[run_id]
    plt.figure(figsize=(12, 12))
    plt.xlabel('Conditions')
    plt.ylabel('TRs')
    plt.imshow(design_matrix)

interactive(children=(IntSlider(value=2, description='run_id', max=4), Output()), _dom_classes=('widget-intera…

In [21]:
# Load previous results
runs_path = dataset_path / f'../glmsingle/'
previous_runs = [p.name for p in runs_path.iterdir()]
print(previous_runs)

run_name = 'run2'
output_path = runs_path / run_name
results_glmsingle = load_results(output_path)

['run1', 'run2']


In [ ]:
# Or run GLMsingle from scratch

#break # set a new run name before running to avoid overwriting old results
run_name = 'run_4'

glmsingle_obj = GLM_single(dict(
    wantlibrary=1,
    wantglmdenoise=1,
    wantfracridge=1,
    wantfileoutputs=[1,1,1,1],
    wantmemoryoutputs=[1,1,1,1],
))

pprint(glmsingle_obj.params)

output_path = dataset_path / f'derivatives_TC2See_prdgm/glmsingle/{run_name}'
results_glmsingle = glmsingle_obj.fit(
    design=design_batch,
    data=fmri_batch,
    stimdur=8,
    tr=tr,
    outputdir=str(output_path),
)

In [ ]:
# Save nifti files for results
save_results(results_glmsingle, output_path, fmri_mask)

In [22]:
# Inspect the results at different layers
@interact(d_layer=(0, D-1))
def plot_results_layer(d_layer):
    plot_results(results_glmsingle, fmri_mask, d_layer)

interactive(children=(IntSlider(value=37, description='d_layer', max=74), Output()), _dom_classes=('widget-int…